In [1]:
!pip install selenium
!pip install webdriver-manager

!pip install requests
!pip install pandas

In [2]:
#  Importar librerías para tratamiento de datos
# -----------------------------------------------------------------------
import pandas as pd  # Pandas es una poderosa librería para manipulación y análisis de datos en Python.

# Importar librerías para web scraping y manipulación de datos
# -----------------------------------------------------------------------
from bs4 import BeautifulSoup
import requests

# Importar librerías para automatización de navegadores web con Selenium
# -----------------------------------------------------------------------
from selenium import webdriver  # Selenium es una herramienta para automatizar la interacción con navegadores web.
from webdriver_manager.chrome import ChromeDriverManager  # ChromeDriverManager gestiona la instalación del controlador de Chrome.
from selenium.webdriver.common.keys import Keys  # Keys es útil para simular eventos de teclado en Selenium.
from selenium.webdriver.support.ui import Select  # Select se utiliza para interactuar con elementos <select> en páginas web.
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as ExpectedCond
# Importar librerías para pausar la ejecución
# -----------------------------------------------------------------------
from time import sleep  # Sleep se utiliza para pausar la ejecución del programa por un número de segundos.
from tqdm import tqdm, trange
import re
# Configuraciones
# -----------------------------------------------------------------------
pd.set_option('display.max_columns', None)  # Establece una opción de Pandas para mostrar todas las columnas de un DataFrame.

In [26]:
def tuplas_actores(id_pelicula):
    #accedemos a la página IMDB:
    #accedemos a la página IMDB:
    driver = webdriver.Chrome() #abrir el navegador
    driver.get("https://www.imdb.com/") # abrir pagina web
    driver.maximize_window() #maximizar ventana navegador
    driver.find_element("css selector", "#__next > div > div > div.sc-jrcTuL.bPmWiM > div > button.icb-btn.sc-bcXHqe.sc-dkrFOg.sc-iBYQkv.dcvrLS.ddtuHe.dRCGjd").click() # acepto las coockies
    sleep(2)
    #-------------------------------------------------------------- ok
    #Buscamos pelicula por el id, en la barra buscador:

    busqueda = driver.find_element("css selector","#suggestion-search").send_keys(id_pelicula , Keys.ENTER)
    sleep(3)

    #--------------------------------------------------------------- ok
        
    #ACCEDEMOS A 'REPARTO PRINCIPAL >'
    driver.find_element("xpath", '//*[@id="__next"]/main/div/section[1]/section/div[3]/section/section/div[1]/div/div[2]/button/span').click()
    sleep(2)
    driver.find_element("xpath", '/html/body/div[4]/div[2]/div/div[2]/div/div[2]/div/div[1]/ul/li[2]/a').click()
    sleep(5)
    

    #--------------------------------------------------------------- ok

    lista_actores_peli_i=[]
    #ciclo for para los 10 primeros actores: range(1,12):
    for i in range(2,12):

        #accedemos a la informacion del actor/actriz:
        
        driver.find_element("xpath", f'//*[@id="fullcredits_content"]/table[3]/tbody/tr[{i}]/td[2]/a').click()
        sleep(5)
        
        #-----------------------------------------------------------------ok

        #seleccionamos MENU desplegable, para acceder a biografia y premios:
        driver.find_element("xpath", '//*[@id="__next"]/main/div/section[1]/section/div[3]/section/section/div[1]/div/div[2]/button/span').click()#acceso a la biografia
        sleep(2)
        #acceso a la biografia
        driver.find_element("xpath", '/html/body/div[4]/div[2]/div/div[2]/div/div[2]/div/div[1]/ul/li[2]/a').click()
        sleep(2)
        #----------------------------------------------------------------ok
        
        # obtenemos el NOMBRE del actor
        nombre= driver.find_element("xpath",'//*[@id="__next"]/main/div/section/section/div[3]/section/section/div[2]/hgroup/h2').text
        sleep(5)
        #---------------------------------------------------------------- ok
        #obtenemos AÑO
        try:
            año_sin_limp=driver.find_element("xpath",'//*[@id="born"]/div/div/div/a[1]').text
            año=(re.findall('\d{4}', año_sin_limp))[0] #sacamos del texto cadena de texto con 4 digitos seguidos, con regex.
        except:
            año='NULL'
        sleep(3)

        #---------------------------------------------------------------- ok

        #obtenemos el texto para filtrar informacion 'CONOCIDO POR':
        try:
            # scroll hasta el texto que necesitamos
            xpath_d='//*[@id="mini_bio"]'
            pyhton_d= driver.find_element(By.XPATH, xpath_d)
            driver.execute_script('arguments[0].scrollIntoView(true)', pyhton_d)
            sleep(6)
            #Obtenemos el texto:
            driver.find_element("xpath", '//*[@id="mini_bio"]').click()
            texto_actor = driver.find_element("xpath",'//*[@id="mini_bio_0"]/div/ul/div/div').text
            print(texto_actor)#XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX
            sleep(6)
            conocido_por=[]
            if re.search('conocido por|conocida por', texto_actor) == None:
                    conocido_por='NULL'
            else:
                patron_a='\.'
                patron_b='conocido por|conocida por' # siempre me va a dar un len()=2, por que en la cadena solo hay un 'conocido por' y este siempre estara en [1]
                patron_c='\(\d{4}'
                patron_d="\), |\) y |\)"
                lista_sep=re.split(patron_b, texto_actor)
                lista_sep_2=re.split(patron_a, lista_sep[1])
                # la estring que nos interesa termina con un '.' simpre va a estar en [0] por que es consecutivo al 'conocido por'
                lista_sep_3=re.split(patron_c, lista_sep_2[0])
                    
                for y in lista_sep_3:
                    lista_sep_3_sin_espacios=(re.sub(patron_d,'', y)).strip() #quitamos espacios y parentesis
                    if lista_sep_3_sin_espacios != '': #quitamos de la lista posibles espacios
                        conocido_por.append(lista_sep_3_sin_espacios)
        except:
            conocido_por='NULL'
        #---------------------------------------------------------------- ok
        sleep(2)
        driver.back() # al acceder a 'mi biografía' hacemos click. tenemos que volver atrás. SEGUIMOS DENTRO DE BIOGRAFIA ACTOR
        sleep(2)
        driver.back() # VOLVEMOS A PAGINA PRINCIPAL ACTOR.
        sleep(10)

        #--------------------------------------------------------------- ok
        #acceso a los PREMIOS, desde el desplegable.
        try:
            driver.find_element("xpath", '//*[@id="__next"]/main/div/section[1]/section/div[3]/section/section/div[1]/div/div[2]/button/span').click()#acceso a la biografiasleep(5)
        
        except:
            #si se sale de la pagina, (si no hay texto, conocido por) volvemos a ella:
            driver.find_element("xpath", f'//*[@id="fullcredits_content"]/table[3]/tbody/tr[{i}]/td[2]/a').click() #acceso actor.
            sleep(2) 
            driver.find_element("xpath", '//*[@id="__next"]/main/div/section[1]/section/div[3]/section/section/div[1]/div/div[2]/button/span').click()#acceso a los premios
            sleep(2)

   
        driver.find_element("xpath", '/html/body/div[4]/div[2]/div/div[2]/div/div[2]/div/div[1]/ul/li[4]/a').click()#acceso a la premios.
        sleep(6)
        #creamos ciclo for para sacar las los premios:
        
        try:
            premios=[]
            for i in range(1,6): #solo queremos 5 pelis
                premio_sin_limpiar=driver.find_element("xpath",f'//*[@id="__next"]/main/div/section/div/section/div/div[1]/section[{i}]').text
                premio_limpio= re.sub('\n',' ', premio_sin_limpiar)
                premios.append(premio_limpio)
            sleep(6)
        except:
            premios='NULL'
        
         

        driver.back()
        sleep(3)

        #--------------------------------------------------------------- ok
        #extraemos informacion que hace.

        que_hace=[]
        try:
            try:
                for z in range(1,4):
                    peli_hace=driver.find_element("xpath",f'//*[@id="__next"]/main/div/section[1]/section/div[3]/section/section/div[2]/div/ul/li[{z}]').text
                    que_hace.append(peli_hace)
                    sleep(2)
            except:
                    peli_hace=driver.find_element("xpath",'//*[@id="__next"]/main/div/section[1]/section/div[3]/section/section/div[2]/div/ul/li').text
                    que_hace.append(peli_hace)
                    sleep(2)
        except:
            que_hace='NULL'
        
        #--------------------------------------------------------------- ok

        #sacamos tupla de cada actor
        lista_infor_actor=[nombre, año, que_hace, conocido_por,premios]
        tupla_x=tuple(lista_infor_actor)
        lista_actores_peli_i.append(tupla_x)#apendeamos a la lista de actores pelicula

        sleep(6)
        driver.back()
        #volvemos a la página principal 

    return lista_actores_peli_i #obtenemos una lista con tuplas informativas de los 10 principales actores.


In [6]:
#esto cambia segun la ruta donde este guardado el csv

datos_accion_csv =pd.read_csv(r"C:\Users\marg_\Desktop\Proyecto_lesion2\project-da-promo-H-module-2-team-Interstellar\API\CSVs\Películas_['Action']_2010-2024.csv") 
datos_drama_cvs = pd.read_csv(r"C:\Users\marg_\Desktop\Proyecto_lesion2\project-da-promo-H-module-2-team-Interstellar\API\CSVs\Películas_['Comedy']_2010-2025.csv")
datos_comedia_cvs = pd.read_csv(r"C:\Users\marg_\Desktop\Proyecto_lesion2\project-da-promo-H-module-2-team-Interstellar\API\CSVs\Películas_['Drama']_2010-2025.csv")


In [7]:
id_accion = datos_accion_csv["ID_IMDB"]
id_drama = datos_drama_cvs["ID_IMDB"]
id_comedia = datos_comedia_cvs["ID_IMDB"]

In [8]:
#convierto en listas

lista_id_accion = id_accion.to_list()
lista_id_comedia = id_comedia.to_list()
lista_id_drama = id_drama.to_list()

In [9]:
#  todos los id en una lista

union_listas =[]
union_listas.extend(lista_id_accion)
union_listas.extend(lista_id_comedia)
union_listas.extend(lista_id_drama)

In [10]:
# cuento numero de elementos en la lista
print(len(union_listas))

15688


In [11]:
# elimino id duplicados duplicados

lista_id_set = set(union_listas)
print(len(lista_id_set))

14544


In [12]:
#lo convierto en lista

lista_id = list(lista_id_set)
print(lista_id)

['tt10008900', 'tt10189138', 'tt1821478', 'tt12975344', 'tt3433996', 'tt11772940', 'tt3291844', 'tt2825240', 'tt3838992', 'tt0359113', 'tt0453562', 'tt0892062', 'tt30248546', 'tt10085338', 'tt3853758', 'tt26470192', 'tt10001728', 'tt10319676', 'tt10508258', 'tt2602732', 'tt1817683', 'tt11332860', 'tt11269026', 'tt10381288', 'tt2376726', 'tt10302296', 'tt10815396', 'tt14497770', 'tt10923334', 'tt13190578', 'tt3289452', 'tt13506800', 'tt11873084', 'tt24640480', 'tt28681424', 'tt1652361', 'tt11433556', 'tt10075032', 'tt11723620', 'tt29229431', 'tt30318139', 'tt2167202', 'tt11460512', 'tt3719968', 'tt13344828', 'tt11781806', 'tt10476718', 'tt1801128', 'tt1059836', 'tt10613286', 'tt10477470', 'tt30331894', 'tt23635684', 'tt1639385', 'tt3345952', 'tt13293926', 'tt10472290', 'tt3840038', 'tt23554170', 'tt10019108', 'tt30421930', 'tt12821606', 'tt3863892', 'tt10370402', 'tt2531120', 'tt10431512', 'tt1791679', 'tt26736605', 'tt10033652', 'tt10167142', 'tt10366902', 'tt11880722', 'tt13894548', '

In [27]:
lista_prueba= lista_id[0]
lista_prueba

'tt10008900'

In [25]:
dic_peli={'id_peli':[], 'actores_10':[]}
no_peli=[]
for id_peli in tqdm(lista_prueba):
    try:
        tupla_una_peli=tuplas_actores(id_peli)
        print(tupla_una_peli)
        dic_peli['actores_10'].append(tupla_una_peli)
        dic_peli['id_peli'].append(id_peli)
    except:
        no_peli.append(id_peli)
        pass

    
print(dic_peli)

 25%|██▌       | 1/4 [01:32<04:38, 92.67s/it]

[('Victoria Crawford', 'NULL', ['Reparto', 'Equipos adicionales', 'Reparto'], ['su papel en Remnants', 'Folly of Man', 'In My Shoes'], 'NULL')]


 75%|███████▌  | 3/4 [03:46<01:18, 78.68s/it]

[('Joseph Vijay', '1974', ['Reparto', 'Música', 'Producción'], ['Mersal', 'Thuppakki', 'Master'], ['Filmfare Awards South 2019 Nominado Filmfare Award - Tamil Film Industry Best Actor Sarkar 2018 Nominado Filmfare Award - Tamil Film Industry Best Actor Mersal 2017 Nominado Filmfare Award - Tamil Film Industry Best Actor Theri 2016 Nominado Filmfare Award - Tamil Film Industry Best Male Playback Singer Puli For the song "Yaendi Yaendi" 2015 Nominado Filmfare Award - Tamil Film Industry Best Actor Kaththi 4 más', 'South Indian International Movie Awards 2022 Nominado SIIMA - Tamil Best Actor in a Leading Role Master 2020 Nominado SIIMA - Tamil Best Actor in a Leading Role Bigil 2019 Nominado SIIMA - Tamil Best Actor in a Leading Role Sarkar 2018 Nominado SIIMA - Tamil Best Actor in a Leading Role Mersal 2017 Nominado SIIMA - Tamil Best Actor in a Leading Role Theri 3 más', 'Vijay Awards 2018 Ganador Favorite Award Favorite Hero Mersal 2018 Nominado Jury Award Best Actor Mersal 2015 Ganad

100%|██████████| 4/4 [05:20<00:00, 80.24s/it]

[('Gemma Soldati', 'NULL', ['Reparto', 'Dirección', 'Guion'], ['su papel en The Adventures of Sleepyhead', 'Speaking', 'Turkey Fools Day'], 'NULL')]
{'id_peli': ['tt10008900', 'tt1821478', 'tt12975344'], 'actores_10': [[('Victoria Crawford', 'NULL', ['Reparto', 'Equipos adicionales', 'Reparto'], ['su papel en Remnants', 'Folly of Man', 'In My Shoes'], 'NULL')], [('Joseph Vijay', '1974', ['Reparto', 'Música', 'Producción'], ['Mersal', 'Thuppakki', 'Master'], ['Filmfare Awards South 2019 Nominado Filmfare Award - Tamil Film Industry Best Actor Sarkar 2018 Nominado Filmfare Award - Tamil Film Industry Best Actor Mersal 2017 Nominado Filmfare Award - Tamil Film Industry Best Actor Theri 2016 Nominado Filmfare Award - Tamil Film Industry Best Male Playback Singer Puli For the song "Yaendi Yaendi" 2015 Nominado Filmfare Award - Tamil Film Industry Best Actor Kaththi 4 más', 'South Indian International Movie Awards 2022 Nominado SIIMA - Tamil Best Actor in a Leading Role Master 2020 Nominado 

In [ ]:
dic_peli={'id_peli':[], 'actores_10':[]}
no_peli=[]
for id_peli in tqdm(lista_prueba):
    try:
        tupla_una_peli=tuplas_actores(id_peli)
        print(tupla_una_peli)
        dic_peli['actores_10'].append(tupla_una_peli)
        dic_peli['id_peli'].append(id_peli)
    except:
        no_peli.append(id_peli)
        pass

    
print(dic_peli)

In [13]:
dic_peli={'id_peli':[], 'actores_10':[]}
for id_peli in tqdm(lista_id):
    
    tupla_una_peli=tuplas_actores(id_peli)
    print(tupla_una_peli)
    dic_peli['actores_10'].append(tupla_una_peli)
    dic_peli['id_peli'].append(id_peli)
    
print(dic_peli)

  0%|          | 1/14544 [01:52<455:07:19, 112.66s/it]

[('Victoria Crawford', 'NULL', ['Reparto', 'Equipos adicionales', 'Reparto'], 'NULL', 'NULL')]


  0%|          | 1/14544 [02:17<555:51:00, 137.60s/it]


NoSuchElementException: Message: no such element: Unable to locate element: {"method":"xpath","selector":"//*[@id="fullcredits_content"]/table[3]/tbody/tr[2]/td[2]/a"}
  (Session info: chrome=124.0.6367.203); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#no-such-element-exception
Stacktrace:
	GetHandleVerifier [0x00007FF72B251522+60802]
	(No symbol) [0x00007FF72B1CAC22]
	(No symbol) [0x00007FF72B087CE4]
	(No symbol) [0x00007FF72B0D6D4D]
	(No symbol) [0x00007FF72B0D6E1C]
	(No symbol) [0x00007FF72B11CE37]
	(No symbol) [0x00007FF72B0FABBF]
	(No symbol) [0x00007FF72B11A224]
	(No symbol) [0x00007FF72B0FA923]
	(No symbol) [0x00007FF72B0C8FEC]
	(No symbol) [0x00007FF72B0C9C21]
	GetHandleVerifier [0x00007FF72B5541BD+3217949]
	GetHandleVerifier [0x00007FF72B596157+3488183]
	GetHandleVerifier [0x00007FF72B58F0DF+3459391]
	GetHandleVerifier [0x00007FF72B30B8E6+823622]
	(No symbol) [0x00007FF72B1D5FBF]
	(No symbol) [0x00007FF